# Skip (residual) connections
In PyTorch, **skip connections** (also called **residual connections**) are usually implemented by adding the input of a layer (or block) back to its output. This is common in ResNets and many modern architectures.

Here’s the general pattern:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# A simple convolutional NN with a skip connection
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()

        # Define a simple 2-layer convolutional neural network with batch normalization
        # between layers
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # if input and output channel dimensions differ, project with 1x1 conv
        # Don't worry too much about this - if you are creating a fixed-size NN and not a generic block you will
        # re-size your tensors directly by hand
        self.shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        # Before running the convolutions capture the input, x, into 'identity'
        # self.shortcut(x) just runs an initial 2d convolution to match input and output sizes
        identity = self.shortcut(x)   # skip connection path

        # Now run the convolutional and batch normalization layers
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        # Here's the important part, just append the original input (identity) to the end of the
        # convolutional outputs (the shortcut(x) convolution ensures that they are the same shape)
        out += identity    # The skip connection!          
        return F.relu(out) # Return the convolutional output + the original data
